# M3 Agentic AI - Email assistant workflow

## 1. Introduction

### 1.1 Lab overview
**email assistant agent**. 

This agentic workflow can carry out various tasks related to email management, including sending emails, searching for emails from a specific sender, and deleting emails. You’ll give it natural language instructions - like “check unread emails from my boss” or “delete the Happy Hour email” - and see how it selects the right tools and completes the task for you.

<img src="lab_overview.png" alt="Example of a calendar assistant" width="700"/>

### 🎯 1.2 Learning outcome
By the end of this lab, you will be able to **connect an LLM to tools**, give natural language instructions, and observe how the agent selects, executes, and validates multi-step tasks such as searching, sending, and deleting emails.


## 2. Initialize environment and client

In [ ]:
# ================================
# Imports
# ================================

# --- Third-party ---
from dotenv import load_dotenv
# import aisuite as ai
import json
import pprint

# --- Local / project ---
import utils
import display_functions
import email_tools

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

# ================================
# Environment & Client
# ================================
load_dotenv()          # Load environment variables from .env


True

## 3. Simulated email service

### 3.1 Components
This lab uses a **simulated email backend** to mimic real-world email interactions.
Think of it as your personal sandbox email inbox: it comes preloaded with messages so you can practice without sending real emails.  


| Layer                   | Purpose                        |
|-------------------------|--------------------------------|
| **FastAPI**             | Exposes REST endpoints         |
| **SQLite + SQLAlchemy** | Stores and queries emails locally |
| **Pydantic**            | Ensures inputs and outputs are valid |
| **AISuite tools**       | Bridge between the LLM and the service |


### 3.2 Endpoints
The service provides several routes that simulate common email actions.  

- `POST /send` → send a new email  
- `GET /emails` → list all emails  
- `GET /emails/unread` → show only unread emails  
- `GET /emails/{id}` → fetch a specific email by ID  
- `GET /emails/search?q=...` → search emails by keyword  
- `GET /emails/filter` → filter by recipient or date range  
- `PATCH /emails/{id}/read` → mark an email as read  
- `PATCH /emails/{id}/unread` → mark an email as unread  
- `DELETE /emails/{id}` → delete an email by ID  
- `GET /reset_database` → reset emails to initial state (for testing)  


> 💡 **Key idea:**
> In the next steps, these endpoints will be exposed as Python functions (tools) that the LLM can call—turning raw routes into agent actions.

### 3.3 Endpoint test helpers
This step is your **sanity check** before handing the controls over to the agent.

The `utils.test_*` functions are quick wrappers around the API endpoints. They let you try actions like **send, list, search, filter, mark, and delete** without writing raw HTTP requests.

Each helper has a clear, self-explanatory name.

For example, test:  
- Send a test email  
- Fetch email by ID  
- List all messages  
- Mark email as read or unread  
- Delete email  

In [4]:
# uncomment the line 'utils.test_*' you want to try
# new_email_id = utils.test_send_email()
# _ = utils.test_get_email(new_email_id['id'])

# _ = utils.test_list_emails()
#_ = utils.test_filter_emails(recipient="test@example.com")
#_ = utils.test_search_emails("lunch")
_ = utils.test_unread_emails()
#_ = utils.test_mark_read(new_email_id['id'])
#_ = utils.test_mark_unread(new_email_id['id'])
#_ = utils.test_delete_email(new_email_id['id'])
#_ = utils.reset_database()


## 4. Tool layer for the email agent

### 4.1 Why tools?
Now that the endpoints are working, the next step is to expose them to the LLM as Python functions called **tools**. Each tool wraps a REST route, transforming raw API calls into actions the agent can perform—like list, read, search, send, delete, or toggle read.

> Think of tools as the agent’s **actuators**: you give a natural language instruction (“check unread emails from my boss and send a polite reply”), and the model chooses **which tools** to call and **in what order** to complete the task.

### 4.2 Design hints
- Keep tool **docstrings** short, imperative, and specific to the action.  
- Return **consistent, compact JSON** so the model can chain results.  
- Prefer **one clear responsibility per tool** (single route, single effect).

### 4.3 Available tools
| Tool Function                      | Action                                                                 |
|------------------------------------|------------------------------------------------------------------------|
| `list_all_emails()`                | Fetch all emails, newest first                                         |
| `list_unread_emails()`             | Retrieve only unread emails                                            |
| `search_emails(query)`             | Search by keyword in subject, body, or sender                          |
| `filter_emails(...)`               | Filter by recipient and/or date range                                  |
| `get_email(email_id)`              | Fetch a specific email by ID                                           |
| `mark_email_as_read(id)`           | Mark an email as read                                                  |
| `mark_email_as_unread(id)`         | Mark an email as unread                                                |
| `send_email(...)`                  | Send a new (simulated) email                                                |
| `delete_email(id)`                 | Delete an email by ID                                                  |
| `search_unread_from_sender(addr)`  | Return unread emails from a given sender (e.g., `boss@email.com`)      |

**Note:** find the email_tools.py file from the top menu by navigating to File > Open.

For example, **testing `list_all_emails()` tool:**
```python
    all_emails = email_tools.list_all_emails()
```

Now, let’s try out a few tools that connect to the simulated endpoints to make sure everything is working. 

👉 **Uncomment** the ones you’d like to run, execute the cell, and review the outputs.

In [3]:
# Test sending a new email and fetch it by ID
new_email = email_tools.send_email("test@example.com", "Lunch plans", "Shall we meet at noon?")
content_ = email_tools.get_email(new_email['id'])

# Uncomment the ones you want to try:
#content_ = email_tools.list_all_emails()
#content_ = email_tools.list_unread_emails()
#content_ = email_tools.search_emails("lunch")
#content_ = email_tools.filter_emails(recipient="test@example.com")
#content_ = email_tools.mark_email_as_read(new_email['id'])
#content_ = email_tools.mark_email_as_unread(new_email['id'])
#content_ = email_tools.search_unread_from_sender("test@example.com")
#content_ = email_tools.delete_email(new_email['id'])

utils.print_html(content=json.dumps(content_, indent=2), title="Testing the email_tools")


~ Sending email to: test@example.com with subject: Lunch plans
Recipient test@example.com
Subject: Lunch plans
Body: Shall we meet at noon?
~ Retrieving email with ID: 7


In [3]:
functions = [ # list of tools that the LLM can access
    email_tools.search_unread_from_sender,
    email_tools.list_unread_emails,
    email_tools.search_emails,
    email_tools.get_email,
    email_tools.mark_email_as_read,
    email_tools.send_email
]

In [ ]:
# import inspect
# import re

# def get_function_schema(func):
#     """
#     Dynamically generates a tool schema by pairing Python's inspection 
#     with docstring parsing to capture descriptions for all parameters.
#     """
#     type_map = {
#         str: "string",
#         int: "integer",
#         float: "number",
#         bool: "boolean",
#         list: "array",
#         dict: "object",
#     }

#     # 1. Get Function Metadata
#     sig = inspect.signature(func)
#     doc = func.__doc__ or ""
    
#     # 2. Extract Main Description (first line)
#     main_desc = doc.strip().split("\n")[0]

#     # 3. Parse Parameter Descriptions from "Args:" block
#     # This regex captures: parameter_name (type): description
#     param_docs = {}
#     param_pattern = re.compile(r"(\w+)\s+\([^)]+\):\s+(.*)")
#     for line in doc.split("\n"):
#         match = param_pattern.search(line)
#         if match:
#             name, desc = match.groups()
#             param_docs[name] = desc.strip()

#     # 4. Build Properties and Required lists
#     properties = {}
#     required = []

#     for name, param in sig.parameters.items():
#         # Get JSON type from hint
#         json_type = type_map.get(param.annotation, "string")
        
#         # Build the parameter entry
#         properties[name] = {
#             "type": json_type,
#             "description": param_docs.get(name, f"The {name} parameter.")
#         }

#         # Check if parameter is required (no default value)
#         if param.default == inspect.Parameter.empty:
#             required.append(name)

#     return {
#         "name": func.__name__,
#         "description": main_desc,
#         "parameters": {
#              "type": "object",
#              "properties": properties,
#              "required": required
#         }
#     }

In [ ]:
# function_schemas = [get_function_schema(func) for func in functions]
# utils.print_html(content=json.dumps(function_schemas, indent=2), title="Function Schemas")

In [ ]:
# print(json.dumps(get_function_schema(email_tools.send_email), indent=2))

{
  "name": "send_email",
  "description": "Send an email (simulated). The sender is set automatically by the server.",
  "parameters": {
    "type": "object",
    "properties": {
      "recipient": {
        "type": "string",
        "description": "The email address of the recipient."
      },
      "subject": {
        "type": "string",
        "description": "The subject of the email."
      },
      "body": {
        "type": "string",
        "description": "The message body content."
      }
    },
    "required": [
      "recipient",
      "subject",
      "body"
    ]
  }
}


## 5. Preparing the agent prompt

Helper function called `build_prompt()`: This function wraps the natural language request in a system-style preamble so the LLM:

In [4]:
def build_prompt(request_: str) -> str:
    return f"""
- You are an AI assistant specialized in managing emails.
- You can perform various actions such as listing, searching, filtering, and manipulating emails.
- Use the provided tools to interact with the email system.
- Never ask the user for confirmation before performing an action.
- If needed, my email address is "you@email.com" so you can use it to send emails or perform actions related to my account.

{request_.strip()}
"""

In [5]:
# Test prompt
example_prompt = build_prompt("Delete the Happy Hour email")
utils.print_html(content=example_prompt, title="Example example_prompt")

### 5.3 Resetting the email service

In [16]:
utils.reset_database()

{'message': 'Database reset and emails reloaded'}

## 6. LLM + Email tools

### 6.1 Scenario
> “Check for unread emails from `boss@email.com`, mark them as read, and send a polite follow-up.”

### 6.2 What happen
1. The agent interprets your instruction.  
2. It selects the right tools (`search_unread_from_sender` → `mark_email_as_read` → `send_email`).  
3. It executes each action automatically, without asking for confirmation.

AISuite handles schema exposure, argument binding, execution, and passing results between steps—so you can focus on **what** the agent achieves, not **how** to call the API.

### 6.3 Run it
*What to look for:*
- A clear **tool-call trace** showing which tools were used and the arguments passed.
- A concise **final message** summarizing the actions performed (e.g., “Found 1 unread email, marked as read, sent follow-up”).

In [7]:
from google import genai
from google.genai import types
import os

# Configure the client and tools
# gemini_api_key = os.getenv("GEMINI_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY_2")

client = genai.Client(api_key=gemini_api_key)

In [18]:
email_tools.search_unread_from_sender("alice@work.com")

~ Searching unread emails from sender: alice@work.com


[{'id': 3,
  'sender': 'alice@work.com',
  'recipient': 'you@email.com',
  'subject': 'Lunch?',
  'body': 'Free for lunch today?',
  'timestamp': '2025-12-18T03:02:52.386464',
  'read': False}]

In [20]:
# Configure the client and model
config = types.GenerateContentConfig(
    tools=functions
)
prompt_ = build_prompt("Check for unread emails from alice@work.com, mark them as read, and send a polite follow-up. Respond with a summary of the email content and steps you have done.")

# Make the request
response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents=prompt_,
    config=config,
)

# Print the final, user-facing response
print("~ Agent response:", response.text)

~ Searching unread emails from sender: alice@work.com
~ Marking email as read with ID: 3
~ Sending email
+ Recipient: alice@work.com
+ Subject: Re: Lunch?
+ Body: Hi Alice, I'm sorry, I'm not available for lunch today. Can we do tomorrow instead?
~ Agent response: I found one unread email from alice@work.com with the subject "Lunch?". I have marked it as read and sent a reply asking to reschedule for tomorrow.


In [21]:
email_tools.search_unread_from_sender("alice@work.com")

~ Searching unread emails from sender: alice@work.com


[]